In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1188.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  319.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2993 personas en los últimos 7 días 

Un positivo PCR cada 1528 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] > 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,1188.0,155.0,339.0,510743.0,30.347944,66.373891,35.0
Huelva-Costa,640.0,89.0,204.0,288115.0,30.890443,70.805060,23.0
Huelva (capital),319.0,48.0,94.0,143663.0,33.411526,65.430904,13.0
Condado-Campiña,359.0,50.0,99.0,155057.0,32.246206,63.847488,10.0
Bollullos Par del Condado,51.0,15.0,28.0,14272.0,105.100897,196.188341,5.0
Isla Cristina,61.0,8.0,27.0,21264.0,37.622272,126.975169,3.0
Ayamonte,40.0,10.0,24.0,20946.0,47.741812,114.580349,2.0
San Juan del Puerto,21.0,3.0,4.0,9300.0,32.258065,43.010753,2.0
Almonte,49.0,5.0,8.0,24191.0,20.668844,33.070150,1.0


In [14]:
if tabla[tabla['Diferencia PCR'] > 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Cabezas Rubias,9.0,1.0,7.0,730.0,136.986301,958.904110,0.0
Villalba del Alcor,42.0,2.0,12.0,3338.0,59.916117,359.496705,0.0
Bollullos Par del Condado,51.0,15.0,28.0,14272.0,105.100897,196.188341,5.0
Zalamea la Real,20.0,1.0,6.0,3068.0,32.594524,195.567145,0.0
Aroche,8.0,4.0,6.0,3073.0,130.165962,195.248942,0.0
Palos de la Frontera,26.0,5.0,17.0,11289.0,44.290903,150.589069,0.0
Isla Cristina,61.0,8.0,27.0,21264.0,37.622272,126.975169,3.0
Nerva,20.0,5.0,6.0,5235.0,95.510984,114.613181,0.0
Ayamonte,40.0,10.0,24.0,20946.0,47.741812,114.580349,2.0
